# Using pre-calculated tables & interpolation

For convenience (and to save computation time!), profiles over a range of radii have been computed for the following set of Sersic indices `n` and flattening (or "stretching," if prolate) `invq`:

    n_arr =     np.arange(0.5, 8.1, 0.1)
    invq_arr =  np.array([1., 2., 3., 4., 5., 6., 7., 8., 10., 20., 100.,
                          1.11, 1.25, 1.43, 1.67, 2.5, 3.33, 0.5, 0.67])
                          
These tables can be loaded as follows.

By default, if `table_path` is not specified, then the system variable ``SERSIC_PROFILE_MASS_VC_DATADIR`` (e.g., in your `.bashrc` file) is used.